In [1]:
from importlib import reload

import datasets
import numpy as np
import torch

import src.models.components.feature_extractor_dinov2
import src.models.components.sign_language_net
import src.models.components.spatiotemporal_encoder

In [2]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

first

In [3]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(first['frames'], sampling_rate=25, pad_to_multiple_of=16, return_tensors="pt")

input_values = batched_feature.input_values
attention_mask = batched_feature.attention_mask

print("input_values:", input_values.shape)
print("attention_mask:", attention_mask)

In [4]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoderConfig

config = SpatiotemporalEncoderConfig()

model = SpatiotemporalEncoder(config)
encoded = model(input_values, attention_mask)

encoded

In [29]:
from transformers import PreTrainedTokenizerFast

reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet

model = SignLanguageNet(tokenizer=PreTrainedTokenizerFast(
    pad_token="__PAD__",
    bos_token="__ON__",
    eos_token="__OFF__",
    unk_token="__UNK__",
    tokenizer_file="../src/etc/rwth_phoenix_tokenizer.json"
))

model

In [42]:
out = model.forward(input_values=input_values, attention_mask=attention_mask, labels=torch.LongTensor([[2, 2]]))

print(out.logits.shape)
pred = torch.argmax(out.logits, dim=2)

print(pred)